In [1]:
#Librerias
import pandas as pd
import numpy as np
import os

In [2]:
path = 'Data/'
sports = pd.read_excel(path+'sports.xlsx')

In [3]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament
0,2.20,2000-08-19,H,1,Football,E0
1,1.60,2000-08-19,H,1,Football,E0
2,2.20,2000-08-19,H,0,Football,E0
3,1.80,2000-08-19,H,0,Football,E0
4,1.55,2000-08-19,H,1,Football,E0


In [4]:
#Creamos una función que convierta la cuota en una probabilidad, para así hacer un análisis más sencillo.
#Esta probabilidad será la que tiene de ganar el favorito

def prob(a):
    return (1/a)*100

In [5]:
sports['Probabilidad'] = sports['CuotaFavorito'].apply(prob)

In [6]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad
0,2.20,2000-08-19,H,1,Football,E0,45.454545
1,1.60,2000-08-19,H,1,Football,E0,62.500000
2,2.20,2000-08-19,H,0,Football,E0,45.454545
3,1.80,2000-08-19,H,0,Football,E0,55.555556
4,1.55,2000-08-19,H,1,Football,E0,64.516129


In [7]:
sports.dtypes

CuotaFavorito           float64
Date             datetime64[ns]
Favorito                 object
GanaFavorito              int64
Sport                    object
Tournament               object
Probabilidad            float64
dtype: object

## El primer análisis que haremos será apostar 10€ siempre al favorito.

In [8]:
sports['Hucha100'] = np.where(sports['GanaFavorito'] == 1, sports['CuotaFavorito']*10-10, -10)

In [20]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha100,HuchaB
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000
2,2.20,2000-08-19,H,0,Football,E0,45.454545,-10.0,-4.545455
3,1.80,2000-08-19,H,0,Football,E0,55.555556,-10.0,-5.555556
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387


In [21]:
sports['Hucha100'].sum()

-163289.0

In [22]:
sports['HuchaB'] = np.where(sports['GanaFavorito'] == 1, (sports['CuotaFavorito']*sports['Probabilidad']/10)-(sports['Probabilidad']/10), -sports['Probabilidad']/10)

In [23]:
sports['HuchaB'].sum()

-92195.14371700516

In [24]:
favoritos = sports[sports['GanaFavorito'] == 1]

In [39]:
favoritos.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha100,HuchaB
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387
6,1.25,2000-08-19,H,1,Football,E0,80.000000,2.5,2.000000
8,1.50,2000-08-19,H,1,Football,E0,66.666667,5.0,3.333333


In [26]:
bb = favoritos.groupby('CuotaFavorito',as_index=False)['GanaFavorito'].sum()

In [28]:
bb.head(10)

,CuotaFavorito,GanaFavorito
0,1.000,96
1,1.001,1
2,1.002,57
3,1.003,1
4,1.005,103
5,1.010,644
6,1.015,2
7,1.020,657
8,1.030,640
9,1.040,739


In [29]:
aa = sports.groupby('CuotaFavorito',as_index=False)['Favorito'].count()

In [31]:
aa.head(10)

,CuotaFavorito,Favorito
0,1.000,96
1,1.001,1
2,1.002,60
3,1.003,1
4,1.005,103
5,1.010,657
6,1.015,2
7,1.020,688
8,1.030,665
9,1.040,783


In [32]:
cc = aa.merge(bb, how='left')

In [34]:
cc.head(10)

,CuotaFavorito,Favorito,GanaFavorito
0,1.000,96,96.0
1,1.001,1,1.0
2,1.002,60,57.0
3,1.003,1,1.0
4,1.005,103,103.0
5,1.010,657,644.0
6,1.015,2,2.0
7,1.020,688,657.0
8,1.030,665,640.0
9,1.040,783,739.0


In [35]:
cc['Porcentaje'] = cc['GanaFavorito']/cc['Favorito']

In [41]:
cc.head(10)

,CuotaFavorito,Favorito,GanaFavorito,Porcentaje,Probabilidad,Indicador
0,1.000,96,96.0,1.000000,100.000000,100.000000
1,1.001,1,1.0,1.000000,99.900100,99.900100
2,1.002,60,57.0,0.950000,99.800399,94.810379
3,1.003,1,1.0,1.000000,99.700897,99.700897
4,1.005,103,103.0,1.000000,99.502488,99.502488
5,1.010,657,644.0,0.980213,99.009901,97.050801
6,1.015,2,2.0,1.000000,98.522167,98.522167
7,1.020,688,657.0,0.954942,98.039216,93.621751
8,1.030,665,640.0,0.962406,97.087379,93.437477
9,1.040,783,739.0,0.943806,96.153846,90.750565


In [37]:
cc['Probabilidad'] = cc['CuotaFavorito'].apply(prob)

In [40]:
cc['Indicador'] = cc['Porcentaje'] * cc['Probabilidad']

In [ ]:
cc['Hucha'] = np.where(sports['GanaFavorito'] == 1, sports['CuotaFavorito']*10-10, -10)

In [42]:
dd = cc[['CuotaFavorito','Indicador']]

In [43]:
dd.head()

,CuotaFavorito,Indicador
0,1.000,100.000000
1,1.001,99.900100
2,1.002,94.810379
3,1.003,99.700897
4,1.005,99.502488


In [45]:
sports = sports.merge(dd, how = 'left')

In [95]:
sports.head()

,CuotaFavorito,Date,Favorito,GanaFavorito,Sport,Tournament,Probabilidad,Hucha100,HuchaB,Indicador,HuchaC
0,2.20,2000-08-19,H,1,Football,E0,45.454545,12.0,5.454545,19.243025,4.233465
1,1.60,2000-08-19,H,1,Football,E0,62.500000,6.0,3.750000,37.083788,5.933406
2,2.20,2000-08-19,H,0,Football,E0,45.454545,-10.0,-4.545455,19.243025,-1.924302
3,1.80,2000-08-19,H,0,Football,E0,55.555556,-10.0,-5.555556,28.955330,-2.895533
4,1.55,2000-08-19,H,1,Football,E0,64.516129,5.5,3.548387,38.679065,5.995255


In [63]:
sports['HuchaC'] = np.where(sports['GanaFavorito'] == 1, sports['CuotaFavorito']*sports['Indicador']/10, -sports['Indicador']/10)

In [64]:
print(sports['Hucha100'].sum())
print(sports['HuchaB'].sum())
print(sports['HuchaC'].sum())

-163289.0
-92195.14371700516
521926.3007665614


In [67]:
ee = sports.groupby('CuotaFavorito')['HuchaC'].sum()

In [84]:
spain = sports[sports['Tournament'] == 'SP1']

In [86]:
spain['HuchaC'].sum()

11938.415176555282

In [87]:
spain['Hucha100'].sum()

-3516.6

In [88]:
spain['HuchaB'].sum()

-1804.004227164421